## Uso YT dlp para la demo, scrapping basico de metadatos. Quiza tengo que pasar a una api oficial para que sea mas estable o tener acceso a mas datos

In [5]:
import yt_dlp
import json
import time

lista_canales = [
    "https://www.youtube.com/@BabyBusES",
    "https://www.youtube.com/@SmileandLearnEspa%C3%B1ol",
    "https://www.youtube.com/@creativoskids",
    "https://www.youtube.com/@JuguetesyColores",
    "https://www.youtube.com/@GenevievesPlayhouse",
    "https://www.youtube.com/@GamingWithGTA00",
    "https://www.youtube.com/@videosdejuguetespawpatrole5118",
]

def scrapear_lista_canales(urls):
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,  # (solo metadatos)
        'playlistend': 10,     # Límite de videos
        'ignoreerrors': True,  
        'no_warnings': True,
        # 'sleep_interval': 1,
    }

    resultados = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        
        for index, url in enumerate(urls):
            
            try:
                # Extraemos la info
                info = ydl.extract_info(url, download=False)
                
                # A veces yt-dlp devuelve None si la url está mal
                if not info:
                    print(f" No se pudo obtener info de {url}")
                    continue

                # Estructuramos los datos
                datos = {
                    "id_canal": info.get('id'),
                    "nombre": info.get('uploader') or info.get('title'),
                    "descripcion": info.get('description', ''),
                    "suscriptores": info.get('channel_follower_count'),
                    "url_origen": url,
                    # Juntamos los títulos de los videos en una lista simple
                    "videos_recientes": []
                }

                # Extraer videos (validando que existan entradas)
                if 'entries' in info:
                    for video in info['entries']:
                        if video and video.get('title'):
                            datos["videos_recientes"].append(video.get('title'))
                
                resultados.append(datos)
                print(f"    OK: {datos['nombre']} ({len(datos['videos_recientes'])} videos)")

            except Exception as e:
                print(f"    Error en este canal: {str(e)}")

    return resultados

# --- EJECUCIÓN ---

start = time.time()
data_scraped = scrapear_lista_canales(lista_canales)
end = time.time()

print(f"\n✨ Proceso terminado en {end - start:.2f} segundos.")

# 2. Guardar en un archivo JSON (Esto es lo que usará tu IA después)
nombre_archivo = "dataset_canales.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data_scraped, f, ensure_ascii=False, indent=4)

print(f"📂 Datos guardados en '{nombre_archivo}'.")

    OK: BabyBus - Canciones Infantiles & Videos para Niños (3 videos)
    OK: Smile and Learn - Español (3 videos)
    OK: Creativos Kids (2 videos)
    OK: Juguetes y Colores (10 videos)
    OK: Genevieve's Playhouse - Learning Videos for Kids (2 videos)
    OK: Gaming With GTA  (10 videos)
    OK: Videos de juguetes Paw Patrol en español (10 videos)

✨ Proceso terminado en 4.43 segundos.
📂 Datos guardados en 'dataset_canales.json'.


## Recordar, meter los embeddings en una base de datos vectorial!!

In [6]:
import json
import time
from sentence_transformers import SentenceTransformer, util

try:
    with open('dataset_canales.json', 'r', encoding='utf-8') as f:
        datos_canales = json.load(f)
    print(f"   --> {len(datos_canales)} canales cargados.")
except FileNotFoundError:
    print(" Error: No encontré el archivo 'dataset_canales.json'. Ejecuta el scraper primero.")
    exit()

# El modelo necesita un solo texto por canal para entender de qué trata.
corpus_textos = []

for canal in datos_canales:
    # Unimos los titulos de videos con comas
    txt_videos = ", ".join(canal.get('videos_recientes', []))
    
    # Creamos el texto maestro para la IA
    # "Nombre: [X]. Descripción: [Y]. Contenido reciente: [Z]"
    texto_full = f"Canal: {canal['nombre']}. Descripción: {canal['descripcion']}. Temas de videos: {txt_videos}"
    
    corpus_textos.append(texto_full)

# --- INDEXACIÓN  ---
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

embeddings_db = model.encode(corpus_textos, convert_to_tensor=True)

def buscar(query_usuario, top_k=3):
    start_time = time.time()
    
    # Vectorizamos lo que escribió el usuario
    query_embedding = model.encode(query_usuario, convert_to_tensor=True)
    
    # Buscamos similitud
    hits = util.semantic_search(query_embedding, embeddings_db, top_k=top_k)
    
    print(f" Búsqueda: '{query_usuario}' ({time.time() - start_time:.4f} seg)")
    
    for hit in hits[0]:
        score = hit['score']
        id_canal = hit['corpus_id'] # El índice en la lista original
        canal_original = datos_canales[id_canal] # Recuperamos el objeto JSON original
        
        # Filtro de calidad
        if score < 0.25: 
            continue
            
        print(f" Match: {score:.2f} | {canal_original['nombre']}")
        # por que hizo match?
        print(f"      (Contexto: {canal_original['videos_recientes'][:2]}...)")
    print("-" * 30)

# --- PRUEBAS ---
# Ahora simulamos las queries libres de tus clientes
buscar("videos")
buscar("aprender")
buscar("juegos")
buscar("tutoriales")
buscar("dibujos animados para bebes")
buscar("niños creativos")
buscar("diversion")
buscar("jugar")

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (c:\Users\Giovanni Pitta\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\__init__.py)


#### *VOLVAMOS A INTENTAR, USANDO LA BASE DE DATOS QUE YA TENEMOS EN KIDSCORP YT*


In [7]:
import os
import pickle  # <--- Librería para guardar archivos
# ... tus otros imports ...

# Nombre del archivo donde guardaremos los "cerebros"
ARCHIVO_EMBEDDINGS = "embeddings_cache.pkl"

# ... (Tu función obtener_datos sigue igual) ...

# --- LÓGICA DE CARGA INTELIGENTE ---
def cargar_o_generar_embeddings(df):
    # 1. ¿Ya existe el archivo guardado?
    if os.path.exists(ARCHIVO_EMBEDDINGS):
        print(f"💾 Cargando embeddings desde '{ARCHIVO_EMBEDDINGS}'...")
        with open(ARCHIVO_EMBEDDINGS, 'rb') as f:
            datos_guardados = pickle.load(f)
            # Verificamos que coincidan con los datos actuales (opcional pero recomendado)
            if len(datos_guardados) == len(df):
                print("✅ Embeddings cargados exitosamente.")
                return datos_guardados
            else:
                print("⚠️ Los datos cambiaron. Recalculando embeddings...")

    # 2. Si no existe o cambiaron los datos, generamos de cero
    print("⚡ Generando nuevos Embeddings (esto tardará)...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    # Asegúrate de haber creado la columna 'texto_para_ia' antes
    embeddings = model.encode(df['texto_para_ia'].tolist(), convert_to_tensor=True)
    
    # 3. Guardamos para la próxima
    print(f"💾 Guardando en '{ARCHIVO_EMBEDDINGS}' para el futuro...")
    with open(ARCHIVO_EMBEDDINGS, 'wb') as f:
        pickle.dump(embeddings, f)
        
    return embeddings

In [ ]:
import os
import psycopg2
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time
from dotenv import load_dotenv

# Carga las variables
load_dotenv(override=True)

def obtener_datos():
    conn = None
    try:
        conn = psycopg2.connect(
            host=os.getenv("DB_HOST"),
            database=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASS"),
            port="5432",
            sslmode="require"
        )
        
        query = """
            SELECT channel_title, channel_description, channel_bs_ch_keywords 
            FROM ods.tbl_canales 
            LIMIT 10000;
        """
        
        df = pd.read_sql_query(query, conn)
        return df

    except Exception as e:
        print(f"❌ Error al conectar: {str(e)}")
        return pd.DataFrame()
    finally:
        if conn: conn.close()

# --- PROCESO PRINCIPAL ---
start = time.time()

# 1. Obtener Datos
df_canales = obtener_datos()

if df_canales.empty:
    print(" No se descargaron datos.")
    exit()

# Rellenamos los NULLs con texto vacío para que no explote la IA
df_canales = df_canales.fillna('')

# 3. CREAR TEXTO ENRIQUECIDO
# Juntamos Título + Descripción + Keywords en una sola columna para la IA
df_canales['texto_para_ia'] = (
    "Canal: " + df_canales['channel_title'] + ". " +
    "Descripción: " + df_canales['channel_description'] + ". " +
    "Keywords: " + df_canales['channel_bs_ch_keywords']
)

# 4. VECTORIZAR
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. En lugar de llamar a model.encode directo, llamas a la función inteligente
embeddings_sb = cargar_o_generar_embeddings(df_canales)

print(f" Proceso terminado en {time.time() - start:.2f} segundos.")
print(f" Canales procesados: {len(df_canales)}")
# --- TEST DE BÚSQUEDA ---
def buscar(query_usuario):
    query_vec = model.encode(query_usuario, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, embeddings_sb, top_k=100)
    
    for hit in hits[0]:
        idx = hit['corpus_id']
        info_canal = df_canales.iloc[idx]
        print(f"   ★ {hit['score']:.2f} | {info_canal['channel_title']}")

# Pruébalo aquí mismo
buscar("videos de manualidades para niños")
buscar("gameplay de minecraft")

📥 Descargando datos...


C:\Users\Giovanni Pitta\AppData\Local\Temp\ipykernel_17016\3801974127.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 487.30it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


⚡ Generando nuevos Embeddings (esto tardará)...


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 664.41it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


💾 Guardando en 'embeddings_cache.pkl' para el futuro...
 Proceso terminado en 275.63 segundos.
 Canales procesados: 10000

🔎 Buscando: 'videos de manualidades para niños'
   ★ 0.79 | KiKi-RiKi Videos Infantiles para niños
   ★ 0.74 | Juntines Planes
   ★ 0.73 | Marie Leiner

🔎 Buscando: 'gameplay de minecraft'
   ★ 0.85 | Pober
   ★ 0.80 | Omarcito
   ★ 0.80 | El baúl de Jaime


In [9]:
buscar("canciones")
buscar("aprender idiomas")
buscar("poopie")
buscar("mrbeast")


🔎 Buscando: 'canciones'
   ★ 0.65 | Canciones De Amor
   ★ 0.64 | Sarah Cabello
   ★ 0.63 | Lyrics Music

🔎 Buscando: 'aprender idiomas'
   ★ 0.62 | Academia do Inglês
   ★ 0.59 | Inglês Fácil com Professor Marcondes
   ★ 0.58 | Listen & Learn English

🔎 Buscando: 'poopie'
   ★ 0.53 | Pocoyo 🇧🇷 Português Brasil - Canal Oficial
   ★ 0.52 | OPPA KÊKI
   ★ 0.51 | Morphle o Meu Pet Magico

🔎 Buscando: 'mrbeast'
   ★ 0.58 | MrAerotech
   ★ 0.51 | MrBeast Gaming en Español
   ★ 0.46 | Mantovani


### Proximos pasos :
#### Mejorar la recopilacion de data. Usar la base de datos que tenemos, y descargar descripciones ultimos 10 videos y si podemos integrar el trafico de cada canal al sistema. Osea, cuantos seguidores tiene (mas asincronico) y cuanto trafico relativo tuvo en el ultimo tiempo asi nos aseguramos de que tengan un buen potencial de generacion de impresiones.
#### Convertir en clases y modularizar.
#### Pensar algun metodo de benchmark.
#### Investigar que hace la funcion semantic search (seguro es cosine similarity pero quiza hay una variante mas util para este problema).
#### Determinar modos de busqueda, osea si van a ser una palabra solo, varias palabras, ambas. Ver lo de la negativa (como hago para notBuscar, funcion aparte o puedo hacerlo todo junto?)(no quiero videos de futbol, por ejemplo)


<span style="color:green">Notas anteriores:</span>

<span style="color:white">Podria hacer un funnel con embeddings - ReRankers, pero creo que para este caso no es muy necesario (se usa para evitar fallas en entender el contexto, el reranker "lee la frase completa")

Quiza pifia con nombre inventados, por ejemplo si alguien le pinta buscar mrbeast puede no salir (solucion hibrida con un tf-idf, para resultados mas exactos)</span>
#### Definir estructura de datos
#### Integrar el grafo que tenemos (?)